In [2]:
#Import Webscraping libraries
from bs4 import BeautifulSoup
import requests
from requests import get

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Import Data structure libraries
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

#Import libraries for controlling crawling rate
from time import sleep, time
from random import randint

from IPython.display import Image
from selenium import webdriver
import time
import re

In [3]:
# name, basic information, all review text
name = []
basic_inform = []
reviewtitle = []
reviewtext = []
reviewdate = []
reviewplace = []
reviewrate = []

In [4]:
length = 0 # current page
main_page = 5
url = "https://www.tripadvisor.com/Restaurants-g60827-Brooklyn_New_York.html" 
options = webdriver.ChromeOptions()
options.set_capability("acceptInsecureCerts", True)
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome('E:\chromedriver_win32/chromedriver',options=options)
driver.implicitly_wait(30)
driver.get(url)
driver.implicitly_wait(100)
time.sleep(10)

<ipython-input-4-bded1ac685e5>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('E:\chromedriver_win32/chromedriver',options=options)


In [22]:
length = 100
while True:
    soup = BeautifulSoup(driver.page_source,'lxml')
    href = soup.find_all('a',{'class':'Lwqic Cj b'})
    for i in range(len(href)):
        length += 1
        print(length)
        if length >= 501:
            break
#         get each restaurant page first person
        # get url and open current restaurant page
        each_href = href[i]['href']
        rest_url = "https://www.tripadvisor.com" + each_href
        print(rest_url)
        restaurant = webdriver.Chrome('E:\chromedriver_win32/chromedriver',options=options)
        restaurant.implicitly_wait(30)
        restaurant.get(rest_url)
        restaurant.implicitly_wait(100)
        time.sleep(5)
        try:
            xpath = "//*[@id='taplc_detail_filters_rr_resp_0']/div/div[1]/div/div[2]/div[4]/div/div[2]/div[1]/div[2]/label/span"
            restaurant.find_element("xpath", xpath).click() # click english languages button
        except:
            restaurant.close()
            restaurant.quit()
            continue
        time.sleep(10)
#       click first reviewer picture button
        soup2 = BeautifulSoup(restaurant.page_source,"lxml")
        pic_but = []
        pic_but = soup2.find('div',{'class':'memberOverlayLink clickable'})
        if pic_but == []:
            length -= 1
            continue
        pic_but = pic_but['id']
        xpath = "//*[@id='" + pic_but + "']/div[1]/div/a/div/div"
        print(xpath)
        try:
            restaurant.find_element("xpath", xpath).click() 
        except:
            print("no such button")
            restaurant.close()
            restaurant.quit()
            continue
#       get html data again
        time.sleep(5)
        soup3 = BeautifulSoup(restaurant.page_source,"lxml")
        people = soup3.find('div',{'class':'info_text pointer_cursor'})
        people = people.text
        # basic information like when join the tripadvisor, age with location
        basic = soup3.find_all('ul', {'class':'memberdescriptionReviewEnhancements'})
        if basic == []:
            continue
        basic = basic[0].find_all('li')
        inform = []
        for j in range(len(basic)):
            inform.append(basic[j].text)
        # get and click url to the people page
        xpath = "//*[@id='BODY_BLOCK_JQUERY_REFLOW']/span/div[3]/div/div/div/a/h3"
        try:
            restaurant.find_element("xpath", xpath).click() 
        except:
            print("fail to click")
            
            continue
        time.sleep(10)
#         now current page is the people's main page
#       click show more button
        xpath = "//*[@id='content']/div[2]/button/div/span[1]"
        try:
            restaurant.find_element("xpath", xpath).click() 
        except:
            print("no show more button")
        time.sleep(5)
#         roll to bottom
        temp_height = 0
        while True:
            restaurant.execute_script("window.scrollBy(0,100000)")
            time.sleep(3)
            check_height = restaurant.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
            if check_height == temp_height:
                break
            temp_height = check_height
#       get text review and rate
        review_title = []
        review_text = []
        review_rate = []
        review_date = []
        review_place = []   
        time.sleep(3)
        soup4 = BeautifulSoup(restaurant.page_source,"lxml")
        data = soup4.find_all('div',{'class','muQub VrCoN'})
        visit_place = soup4.find_all('div',{'class','Jkczm b W o q ui_link'})
        for j in range(len(data)):
            if j >= len(visit_place):
                break
            title = data[j].find_all('div', {'class', 'AzIrY b _a VrCoN'})
            if title == []:
                review_title.append("None")
            else:
                title = title[0].text
                review_title.append(title)
#             
            text = data[j].find_all('q', {'class', 'BTPVX'})
            if text == []:
                review_text.append("None")
            else:
                text = text[0].text
                review_text.append(text)
#             
            b = data[j].find('span')
            rate = b.get('class')[1]
            rate = rate.split("_")[1]
            value = int(rate) / 10
            review_rate.append(value)
#             
            date = data[j].find_all('div',{'class','LmTJN S4 H3 Ci'})
            if date == []:
                review_date.append("None")
            else:
                date = date[0].text
                review_date.append(date)
#             
            place = visit_place[j].text
            review_place.append(place)
            
        # append
        name.append(people)
        basic_inform.append(inform)
        reviewtitle.append(review_title)
        reviewtext.append(review_text)
        reviewdate.append(review_date)
        reviewplace.append(review_place)
        reviewrate.append(review_rate)
        restaurant.close()
        restaurant.quit()
    # click main page next page button
    try:
        if main_page == 1:
            xxpath = "//*[@id='EATERY_LIST_CONTENTS']/div[2]/div/a"
        else:
            xxpath = "//*[@id='EATERY_LIST_CONTENTS']/div[2]/div/a[2]"
            driver.find_element("xpath", xxpath).click() # click next button to next page
            main_page += 1
    except:
        print("you have copy all data from tripadvisor's database")
        break
    time.sleep(10)
    if length >= 501:
        break
        

101
https://www.tripadvisor.com/Restaurant_Review-g60827-d19585282-Reviews-Ten_Hope-Brooklyn_New_York.html


<ipython-input-22-ae79852d9650>:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  restaurant = webdriver.Chrome('E:\chromedriver_win32/chromedriver',options=options)


//*[@id='UID_4D5AD31F8CBBF15FC1841B9ADEC5138E-SRC_852805201']/div[1]/div/a/div/div
no show more button
102
https://www.tripadvisor.com/Restaurant_Review-g60827-d3378202-Reviews-Patrizia_s_of_Williamsburg-Brooklyn_New_York.html
//*[@id='UID_E0837D9B13D2BDB4EA385F7313B22C24-SRC_780283108']/div[1]/div/a/div/div
103
https://www.tripadvisor.com/Restaurant_Review-g60763-d424475-Reviews-Pongsri_Thai_Restaurant-New_York_City_New_York.html
//*[@id='UID_FE1A931A0E58707D44F0D74B2DCBCA7C-SRC_868013744']/div[1]/div/a/div/div
104
https://www.tripadvisor.com/Restaurant_Review-g60763-d2020818-Reviews-The_Crosby_Bar-New_York_City_New_York.html
//*[@id='UID_F3FDFC3CB5DA5ED13DD0917C34ED25AD-SRC_836821754']/div[1]/div/a/div/div
105
https://www.tripadvisor.com/Restaurant_Review-g60763-d555948-Reviews-Great_NY_Noodletown-New_York_City_New_York.html
//*[@id='UID_477A2000CF2B83F16E26CC49A9D004AB-SRC_864026945']/div[1]/div/a/div/div
no show more button
106
https://www.tripadvisor.com/Restaurant_Review-g60763-d

//*[@id='UID_3646B20ED5C659EA5FEC60B0F9570693-SRC_843252972']/div[1]/div/a/div/div
143
https://www.tripadvisor.com/Restaurant_Review-g60827-d19585282-Reviews-Ten_Hope-Brooklyn_New_York.html
//*[@id='UID_4D5AD31F8CBBF15FC1841B9ADEC5138E-SRC_852805201']/div[1]/div/a/div/div
no show more button
144
https://www.tripadvisor.com/Restaurant_Review-g60827-d14199287-Reviews-Oxomoco-Brooklyn_New_York.html
//*[@id='UID_F8C159ED63BBA91209E32380D960E77B-SRC_867699449']/div[1]/div/a/div/div
145
https://www.tripadvisor.com/Restaurant_Review-g60763-d1547929-Reviews-Nam_Son_Vietnamese_Restaurant-New_York_City_New_York.html
//*[@id='UID_E6A379A2B64EBB21B2175E39F3F693AD-SRC_845640358']/div[1]/div/a/div/div
146
https://www.tripadvisor.com/Restaurant_Review-g60763-d1950899-Reviews-Bill_s_Bar_Burger-New_York_City_New_York.html
//*[@id='UID_CF39273E28961B39A1C9956342BAFF4C-SRC_871143396']/div[1]/div/a/div/div
147
https://www.tripadvisor.com/Restaurant_Review-g60763-d1318062-Reviews-Almond-New_York_City_New_Y

//*[@id='UID_980E5BFDE3D6C81F1B6F91CD6F2FE893-SRC_747522290']/div[1]/div/a/div/div
184
https://www.tripadvisor.com/Restaurant_Review-g60763-d12541041-Reviews-Banter-New_York_City_New_York.html
//*[@id='UID_9C6E1696358E35D2E26CAAD8DD4E66B7-SRC_871015353']/div[1]/div/a/div/div
185
https://www.tripadvisor.com/Restaurant_Review-g60827-d11844101-Reviews-ICHIRAN_Brooklyn-Brooklyn_New_York.html
//*[@id='UID_A1E7943028CB5AA3B7F4612E95A28D1C-SRC_839422614']/div[1]/div/a/div/div
186
https://www.tripadvisor.com/Restaurant_Review-g60827-d619947-Reviews-The_Good_Fork-Brooklyn_New_York.html
//*[@id='UID_A58525550A9498AAADF8EDA4E8AACAD1-SRC_686828530']/div[1]/div/a/div/div
187
https://www.tripadvisor.com/Restaurant_Review-g60827-d6028371-Reviews-Caracas_Arepa_Bar-Brooklyn_New_York.html
//*[@id='UID_DCEEC40D0BA1151D8BF618270691B5E5-SRC_836726905']/div[1]/div/a/div/div
188
https://www.tripadvisor.com/Restaurant_Review-g60763-d1109605-Reviews-Bobby_Van_s_Steakhouse_Grill-New_York_City_New_York.html
//*[

...

//*[@id='UID_3E2F948EE9FE8E236E053D00C4B9BD9E-SRC_867017800']/div[1]/div/a/div/div
308
https://www.tripadvisor.com/Restaurant_Review-g60763-d8130715-Reviews-The_Clocktower-New_York_City_New_York.html
//*[@id='UID_DC0A546D16E78C520B5924B17FDDA41D-SRC_869143865']/div[1]/div/a/div/div
309
https://www.tripadvisor.com/Restaurant_Review-g60763-d424307-Reviews-Onieal_s_Restaurant-New_York_City_New_York.html
//*[@id='UID_2AA5F0987147D638B373EAF5ED39651F-SRC_860633519']/div[1]/div/a/div/div
no show more button
310
https://www.tripadvisor.com/Restaurant_Review-g60763-d911311-Reviews-Hill_Country_Barbecue_Market-New_York_City_New_York.html
//*[@id='UID_58FA68B371127580EAF55480B6D9AA5B-SRC_869325437']/div[1]/div/a/div/div
311
https://www.tripadvisor.com/Restaurant_Review-g60827-d2462682-Reviews-The_Islands-Brooklyn_New_York.html
//*[@id='UID_C4DD1AC8925B1D2F58F422B9B20BF66A-SRC_839980682']/div[1]/div/a/div/div
no show more button
312
https://www.tripadvisor.com/Restaurant_Review-g60763-d424501-Rev

//*[@id='UID_0F3890FB23C170329645590631233B62-SRC_829446829']/div[1]/div/a/div/div
349
https://www.tripadvisor.com/Restaurant_Review-g60827-d23422638-Reviews-Em_Vietnamese_Bistro-Brooklyn_New_York.html
//*[@id='UID_5A07DA1E1FDDA6EE413D7D76EA044A12-SRC_862720018']/div[1]/div/a/div/div
no show more button
350
https://www.tripadvisor.com/Restaurant_Review-g60763-d2256306-Reviews-Artichoke_Basille_s_Pizza-New_York_City_New_York.html
//*[@id='UID_CAE19AE29B80DDC0896239CA795DA150-SRC_866095499']/div[1]/div/a/div/div
no show more button
351
https://www.tripadvisor.com/Restaurant_Review-g60763-d7814915-Reviews-Loi_Estiatorio-New_York_City_New_York.html
//*[@id='UID_9B18934269D10DC2C471CD084EC1ABC9-SRC_871108105']/div[1]/div/a/div/div
no show more button
352
https://www.tripadvisor.com/Restaurant_Review-g60763-d17760513-Reviews-Amici_Ristorante-New_York_City_New_York.html
//*[@id='UID_FBF971A7F8B07155F6A1B190EDFEE826-SRC_870882341']/div[1]/div/a/div/div
no show more button
353
https://www.tripa

390
https://www.tripadvisor.com/Restaurant_Review-g60763-d6732496-Reviews-Tutto_Il_Giorno-New_York_City_New_York.html
//*[@id='UID_DEC6F48ED0A9216A06C0A09807573C36-SRC_749522591']/div[1]/div/a/div/div
no show more button
391
https://www.tripadvisor.com/Restaurant_Review-g60763-d5014475-Reviews-Peacefood_Cafe-New_York_City_New_York.html
//*[@id='UID_87C4B75E978EC297899ED5F94D92C4F3-SRC_786075461']/div[1]/div/a/div/div
392
https://www.tripadvisor.com/Restaurant_Review-g48080-d3923887-Reviews-Casa_Enrique-Long_Island_City_Queens_New_York.html
//*[@id='UID_E4A63435EE8936BAB0009D26F4785B3E-SRC_870211954']/div[1]/div/a/div/div
393
https://www.tripadvisor.com/Restaurant_Review-g60763-d425890-Reviews-El_Sombrero_Restaurant-New_York_City_New_York.html
//*[@id='UID_3690D3C5798F1157B7E09B9545362AAC-SRC_859139780']/div[1]/div/a/div/div
no show more button
394
https://www.tripadvisor.com/Restaurant_Review-g60763-d457802-Reviews-Sarge_s_Delicatessen_Diner-New_York_City_New_York.html
//*[@id='UID_FDB

//*[@id='UID_9AA3144B714D6EB99C7909BD14440A95-SRC_605151622']/div[1]/div/a/div/div
432
https://www.tripadvisor.com/Restaurant_Review-g60763-d5039941-Reviews-Bareburger_Hell_s_Kitchen-New_York_City_New_York.html
//*[@id='UID_BB79C20BB110FDAFF65153FDD8BF067D-SRC_867831821']/div[1]/div/a/div/div
433
https://www.tripadvisor.com/Restaurant_Review-g60763-d519498-Reviews-EN_Japanese_Brasserie-New_York_City_New_York.html
//*[@id='UID_C5A8B46934E1CFF5229A6FB04544B803-SRC_867640706']/div[1]/div/a/div/div
no show more button
434
https://www.tripadvisor.com/Restaurant_Review-g60763-d10513201-Reviews-Russ_Daughters_Cafe-New_York_City_New_York.html
//*[@id='UID_483634D598601CB5173CBC3DF52555B5-SRC_747409657']/div[1]/div/a/div/div
435
https://www.tripadvisor.com/Restaurant_Review-g60763-d5002599-Reviews-JG_Melons-New_York_City_New_York.html
//*[@id='UID_79710FA31A950A8062C78AEB42B033CC-SRC_848284018']/div[1]/div/a/div/div
436
https://www.tripadvisor.com/Restaurant_Review-g60827-d1552225-Reviews-Athen

//*[@id='UID_FF9BBA83A79D73021F9CCE4575A29AF7-SRC_868878068']/div[1]/div/a/div/div
473
https://www.tripadvisor.com/Restaurant_Review-g60827-d1636984-Reviews-Fonda-Brooklyn_New_York.html
//*[@id='UID_2834EE69208F6D7AC871EDAF34ADC5E9-SRC_802621959']/div[1]/div/a/div/div
474
https://www.tripadvisor.com/Restaurant_Review-g60763-d479361-Reviews-The_Beekman_Pub-New_York_City_New_York.html
//*[@id='UID_EEF83CC69DAD5982829C70213ADAE9DD-SRC_775996832']/div[1]/div/a/div/div
no show more button
475
https://www.tripadvisor.com/Restaurant_Review-g60763-d424314-Reviews-Takahachi_Tribeca_Restaurant-New_York_City_New_York.html
//*[@id='UID_CE0026B12D6B9AF2FD690DD0DE4BF0A0-SRC_865428766']/div[1]/div/a/div/div
476
https://www.tripadvisor.com/Restaurant_Review-g60763-d1988088-Reviews-Park_Avenue_Tavern-New_York_City_New_York.html
//*[@id='UID_3556C553A4A9BB2077832F1916247709-SRC_861659794']/div[1]/div/a/div/div
477
https://www.tripadvisor.com/Restaurant_Review-g60763-d7221913-Reviews-Cotenna-New_York_Cit

In [61]:
pic_but == 'NoneType'

False

In [55]:
print(len(reviewrate[0]))
print(len(reviewtext[0]))

7
7


In [68]:
length

101

In [17]:
print(len(name))

303


In [18]:
dataframe = { "name" : name,
              "information" : basic_inform,
             "review_title" : reviewtitle,
             "review_text" : reviewtext,
             "review_rate" : reviewrate,
             "review_date" : reviewdate,
             "visit_palce" : reviewplace
    
}

In [19]:
man = pd.DataFrame(dataframe)

In [20]:
man

,name,information,review_title,review_text,review_rate,review_date,visit_palce
0,mrchunky,"[Tripadvisor member since 2009, 35-49 man from...","[Family friendly, Great themed restaurant, Bus...",[Great family meal. Good burgers for the child...,"[5.0, 5.0, 4.0, 4.0, 4.0, 5.0, 2.0, 5.0, 4.0, ...","[Date of visit: August 2022, Date of visit: Ju...","[Poe's Tavern, Mole Mexican Restaurant & Tequi..."
1,stevenbK9629RX,"[Tripadvisor member since 2014, 50-64 man from...","[Manhattan Little Italy, Recent Stay, Angama R...",[It’s so great to have “Angelo’s” back after b...,"[5.0, 5.0, 5.0, 3.0, 1.0, 5.0, 5.0, 5.0, 3.0]","[Date of visit: November 2022, Date of stay: M...","[Casa D'Angelo, Charming Luxury Lodge & Privat..."
2,815jemman,"[Tripadvisor member since 2010, 35-49 woman fr...","[Fantastic food & staff, Nice staff - food poo...",[Having researched Mexican restaurants in NYC ...,"[5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 5.0, ...","[Date of visit: October 2022, Date of visit: A...","[La Contenta, Grivola, Brasserie L'affiche, Pu..."
3,AndrewInJax,"[Tripadvisor member since 2012, From Jacksonvi...","[Being as Respectful as I can..., Top 10 Tacos...",[Planned this family/holiday dinner in NYC for...,"[2.0, 5.0, 5.0, 5.0]","[Date of visit: November 2022, Date of visit: ...","[Craft, Captain's Choice, Betsy's Cabins at Mo..."
4,JoSo67,"[Tripadvisor member since 2008, 35-49 man from...","[A nice BBQ gem in Brooklyn, Small Batch Tour ...",[We always eat at Mable’s before we visit Broo...,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 5.0, 5.0, ...","[Date of visit: September 2022, Date of experi...","[Mable's Smokehouse & Banquet Hall, Brooklyn B..."
...,...,...,...,...,...,...,...
298,maxwell1323,"[Tripadvisor member since 2006, From San Anton...","[Should Have Bought a Frozen Pizza Instead, Ov...",[Got a pizza to go and brought it back to the ...,"[2.0, 3.0, 5.0, 3.0, 4.0, 5.0, 4.0, 4.0, 5.0, ...","[Date of visit: November 2022, Date of visit: ...","[It's Italia, the girl & the fig, Oso Sonoma, ..."
299,200laws,[Tripadvisor member since 2022],"[Love Luv this place, The Best!!!!, Horrible, ...","[What a wonderful time, definitely busy for a ...","[5.0, 5.0, 1.0, 5.0, 5.0]","[Date of visit: December 2022, Date of visit: ...","[Dos Caminos, Dos Caminos, STK Steakhouse, Dos..."
300,therealmeltam,"[Tripadvisor member since 2013, Woman from New...","[Tourist trap, Park is free to enter but orchi...",[Singapore chili crab was at the top of my foo...,"[2.0, 5.0, 5.0, 4.0, 4.0, 2.0, 5.0, 4.0, 5.0, ...","[Date of visit: September 2022, Date of experi...","[JUMBO Seafood (Riverside Point), Singapore Bo..."
301,cgny1fwd,"[Tripadvisor member since 2014, From Houston, ...","[Classic east european, Okay, Mixed, Okay, Pre...",[Counter feet after counter feet of baked good...,"[4.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0, ...","[Date of visit: October 2022, Date of visit: O...","[Syrena Piekarnia, Calexico - Greenpoint, Clas..."


In [21]:
man.to_csv('tripadvisor_eachperson_303.csv')